In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tokenization
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [4]:
%%time
url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/2'
bert_layer = hub.KerasLayer(url, trainable=True)

Wall time: 16.8 s


In [5]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [6]:
def bert_encoder(texts, tokenizer, max_len=512):    
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [7]:
train_input = bert_encoder(train_data.text, tokenizer, max_len=160)
text_input = bert_encoder(test_data.text, tokenizer, max_len=160)
train_labels = train_data.target.values

In [9]:
def build_model(max_len=512):
    input_word_ids = keras.layers.Input(shape=(max_len,), 
                                 dtype=tf.int32, name='positional_ids')
    input_segment_ids = keras.layers.Input(shape=(max_len,), 
                                    dtype=tf.int32, name='segment_ids')
    input_mask = keras.layers.Input(shape=(max_len,), 
                              dtype=tf.int32, name='input_mask')
    pooled_output, sequence_output = bert_layer([input_word_ids, 
                                                 input_mask, 
                                                 input_segment_ids])
    clf_output = sequence_output[:, 0, :]
    output = keras.layers.Dense(1, activation='sigmoid')(clf_output)
    model = keras.Model(inputs=[input_word_ids, input_mask, input_segment_ids], 
                        outputs=output)
    model.compile(optimizer= RMSprop(lr=2e-6), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    return model

In [10]:
model = build_model(max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
positional_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   positional_ids[0][0]             
                                                                 input_mask[0][0]             

In [ ]:
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    batch_size=16
)

Train on 6090 samples, validate on 1523 samples
Epoch 1/3


In [ ]:
test_pred = model.predict(test_input)
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)